# Create & Update AGOL View Layers Based on Jurisdictional Boundaries
## Utah Critical Infrastructure Prioritization (UCIP) Tool -- Utah DEM
This script is intended to manage hosted feature layers associated with the UCIP tool, including creating or deleting view layers filtered to specific geographies as well as updating all associated information with those view layers based on updates to the master layer. Particularly, this script allows updates to view layers of all 350+ jurisdictions in Utah to match new or updated fields in the master layer, symbology, pop-ups as well as form info (for editing or smart forms). 

### Import Libraries and Sign in to active GIS

In [4]:
from arcgis.gis import GIS
from arcgis.gis import ContentManager
from arcgis.features import FeatureLayerCollection
import json
import getpass
import arcpy



# Set variables, get AGOL username and password
portal_url = arcpy.GetActivePortalURL()
print(portal_url)

my_agol = GIS(portal_url,"GIS@UtahDEM")



cm = ContentManager(my_agol)

https://www.arcgis.com/
Enter password: ········


### Assign Master Hosted Feature Layer

In [5]:
# The Hosted Feature Layer containing a statewide dataset (Currently UCIP_Live_Layer)
#ucip_locs = my_agol.content.get('b81c64945f004ef28e9dba448e0a677e')
#ucip_locs = my_agol.content.get('419155dca4844b94b44c68f68b4b4ccb')
ucip_locs = my_agol.content.get('f3e43dcc7fc3495ea8731597b63b690d') #UCIP_Live_Layer


ucip_locs_flc = FeatureLayerCollection.fromitem(ucip_locs)
ucip_locs_layer = ucip_locs.layers[0]
ucip_properties = ucip_locs_layer.properties
ucip_symbology = (ucip_properties["drawingInfo"])



### Assign Boundary/Jurisdictional Layer(s)

In [6]:
# The Hosted Feature Layer containing the regional division, counties and counties + municipalities

counties = my_agol.content.get('5d55fd5a8ad34448a32b2b5e34ce9ab9').layers[0].query() #just counties
counties_munis_gap = my_agol.content.get('c29faaea33da4cb680bf4a0e6a676732').layers[0].query() #gap in counties where municipalities are, plus municipalities
counties_munis = my_agol.content.get('6e5e56892ecb4be5990467419f69d248').layers[0].query()#counties and municipalities overlapping
counties_gaps = my_agol.content.get('f98099791f8745ae8159e27735610b3f').layers[0].query() #just counties with gaps where municipalities are

# Get the Spatial Reference
spat_ref = counties_munis.spatial_reference

#For all items below this, switching counties with county_munis will
#change the geometry to include counties and municipalities'''

### Collect To-Update Info from Master Hosted Feature Layer

In [7]:

fields = ucip_properties.fields

layer_data=ucip_locs.get_data()
drawingInfo = layer_data["layers"][0]["layerDefinition"]["drawingInfo"]
formInfo=layer_data["layers"][0]["formInfo"]
popupInfo=layer_data["layers"][0]["popupInfo"]

print(json.dumps(layer_data))


{"layers": [{"layerDefinition": {"defaultVisibility": true, "definitionExpression": null, "drawingInfo": {"renderer": {"authoringInfo": {"classificationMethod": "esriClassifyManual", "visualVariables": [{"maxSliderValue": 68, "minSliderValue": 9, "theme": "high-to-low", "type": "sizeInfo"}]}, "type": "uniqueValue", "visualVariables": [{"type": "sizeInfo", "valueExpression": "// Write a script that returns a value that will be used to draw features.\r\n// For example, find the percentage of males:\r\n// Round(($feature.MalePop / $feature.TotalPop) * 100, 2)\r\n\r\nNumber($feature[\"Total_Score_Number\"])", "valueExpressionTitle": "TotalScore", "legendOptions": {"title": "", "showLegend": true}, "maxDataValue": 68, "maxSize": 20.25, "minDataValue": 9, "minSize": 3.75}], "field1": "infrastructure_sector", "uniqueValueGroups": [{"classes": [{"label": "Government Facilities", "symbol": {"type": "esriPMS", "angle": 0, "xoffset": 0, "yoffset": 0, "contentType": "image/png", "imageData": "iVBO

### Create or Update View Layers Based on Jurisdiction
This script will create a view layer based on the jurisdiction if it does not already exist. If it does exist, it will update the view layer to match the fields/pop-up info/symbology/form info to match the master hosted feature layer. There is also a section to check whether the layer has been recently updated (in case the script stops and has to restart). 

In [27]:
import datetime

# Loop through the regional division to create the views or update existing views for ALL JURISDICTIONS
for index, county in enumerate(counties_munis):
    county_name = counties_munis.features[index].attributes['NAME']
    print(county_name)
    view_name = 'UCIP_' + county_name + "_LIVE_View"
    print(view_name)
    
    # Get the geometry for the regions
    view_geom = counties_munis.features[index].geometry.get('rings')
    name_avail = cm.is_service_name_available(service_name = view_name,service_type = "featureService")
    
    # Check if view exists
    if  name_avail == True:
        new_view = ucip_locs_flc.manager.create_view(name=view_name)
        # Search for newly created View
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)

        service_layer = view_flc.layers[0]
        layerTags = [county_name,view_name,"UCIP","View Layer","Live"]

        # Populate the update_dict with the geometry and the spatial reference
        update_dict = {"tags":layerTags,"viewLayerDefinition":{"filter":
                                                               {"operator":"esriSpatialRelContains","value":
                                                                {"geometryType":"esriGeometryPolygon","geometry":
                                                                 {"rings": view_geom,
                                                                  "spatialReference":spat_ref}}}}}
        
        # Assign category to new view layer
        service_layer_id=view_search.id
        my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                     "categories": ["/Categories/UCIP"]}}])
        
        # Update the definition to include the Area of Interest
        service_layer.manager.update_definition(update_dict)
        
        # Move layer to UCIP live folder
        view_search.move(folder="UCIP - LIVE")

        print("Added ",view_name)
    
    else:
        print(view_name," Exists")
        
        # Search for newly View
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)
        update_date = datetime.date.fromtimestamp(view_search.modified/1000)
        print(update_date)
        print(datetime.datetime.today())
        
        if update_date == datetime.date.today():
            print("Recently Updated")
        else:

            service_layer = view_flc.layers[0]
            layerTags = [county_name,view_name,"UCIP","View Layer"]


            # Populate the update_dict with the geometry and the spatial reference
            dict_fields = [{"name":f"{f.name}","visible":True} for f in fields]
            #print(dict_fields)
            view_layer = view_search.get_data()
            view_layer["layers"][0]["popupInfo"]=popupInfo
            view_layer["layers"][0]["formInfo"]=formInfo
            view_layer["layers"][0]["layerDefinition"]["drawingInfo"]=drawingInfo

            #Update the layer tags and fields
            update_dict_fields = {"tags":layerTags,"fields":dict_fields}
            service_layer.manager.update_definition(update_dict_fields)
            view_search.update(update_dict_fields)

            # Update the layer symbology, pop-up info and form-info
            update_dict = {}
            update_dict = {"text":json.dumps(view_layer)}
            view_search.update(update_dict)

            # Assign category to new view layer
            service_layer_id=view_search.id
            my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                         "categories": ["/Categories/UCIP"]}}])

            # Add layer to UCIP_LIVE folder
            view_search.move(folder="UCIP - LIVE")

            print("Updated ",view_name)

print('Done!')


Nephi
UCIP_Nephi_LIVE_View
UCIP_Nephi_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:20.539038
Recently Updated
Rockville
UCIP_Rockville_LIVE_View
UCIP_Rockville_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:21.176864
Recently Updated
Woodruff
UCIP_Woodruff_LIVE_View
UCIP_Woodruff_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:21.839085
Recently Updated
Sigurd
UCIP_Sigurd_LIVE_View
UCIP_Sigurd_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:22.545523
Recently Updated
Cleveland
UCIP_Cleveland_LIVE_View
UCIP_Cleveland_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:23.258858
Recently Updated
South Ogden
UCIP_South Ogden_LIVE_View
UCIP_South Ogden_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:23.979484
Recently Updated
Wendover
UCIP_Wendover_LIVE_View
UCIP_Wendover_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:24.686414
Recently Updated
Richmond
UCIP_Richmond_LIVE_View
UCIP_Richmond_LIVE_View  Exists
2023-12-11
2023-12-11 08:32:25.263595
Recently Updated
Riverdale
UCIP_Riverdale_LIVE_View
UCIP_

### Create layers for each county to be used for VIEW ONLY (Not Editing)
*Needs to be updated to reflect symbology/pop-ups/form info/fields

In [6]:
# Loop through the regional division to create the views COUNTIES ONLY VIEW ONLY
for index, county in enumerate(counties):
    county_name = counties.features[index].attributes['NAME']
    print(county_name)
    view_name = 'UCIP_' + county_name + "_LIVE_ViewONLY"
    print(view_name)
    
    # Get the geometry for the regions
    view_geom = counties.features[index].geometry.get('rings')
    name_avail = cm.is_service_name_available(service_name = view_name,service_type = "featureService")
    
    # Check if view exists
    if  name_avail == True:
        new_view = ucip_locs_flc.manager.create_view(name=view_name)
        
        # Search for newly created View
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)

        service_layer = view_flc.layers[0]
        layerTags = [county_name,view_name,"UCIP","View Layer","Live"]


        
        # Populate the update_dict with the geometry and the spatial reference
        update_dict = {"tags":layerTags,"viewLayerDefinition":{"filter":
                                                               {"operator":"esriSpatialRelContains","value":
                                                                {"geometryType":"esriGeometryPolygon","geometry":
                                                                 {"rings": view_geom,
                                                                  "spatialReference":spat_ref}}}}}
        
        # Assign category to new view layer
        service_layer_id=view_search.id
        my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                     "categories": ["/Categories/UCIP"]}}])
                

        # Update the definition to include the Area of Interest
        service_layer.manager.update_definition(update_dict)
        
        # Move layer to UCIP live folder
        view_search.move(folder="UCIP - LIVE")

        print("Created ",view_name)
        
    else:
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)

        service_layer = view_flc.layers[0]
        layerTags = [county_name,view_name,"UCIP","View Layer","Live"]

        # Populate the update_dict with the geometry and the spatial reference
        update_dict = {"tags":layerTags,"viewLayerDefinition":{"filter":
                                                               {"operator":"esriSpatialRelContains","value":
                                                                {"geometryType":"esriGeometryPolygon","geometry":
                                                                 {"rings": view_geom,
                                                                  "spatialReference":spat_ref}}}}}
        
        # Assign category to new view layer
        service_layer_id=view_search.id
        my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                     "categories": ["/Categories/UCIP"]}}])
        

        # Update the definition to include the Area of Interest
        service_layer.manager.update_definition(update_dict)
        
        # Move layer to UCIP live folder
        view_search.move(folder="UCIP - LIVE")

        print("Updated ",view_name)

print('Done!')


CACHE
UCIP_CACHE_LIVE_ViewONLY
Updated  UCIP_CACHE_LIVE_ViewONLY
DUCHESNE
UCIP_DUCHESNE_LIVE_ViewONLY
Updated  UCIP_DUCHESNE_LIVE_ViewONLY
DAVIS
UCIP_DAVIS_LIVE_ViewONLY
Updated  UCIP_DAVIS_LIVE_ViewONLY
MILLARD
UCIP_MILLARD_LIVE_ViewONLY
Updated  UCIP_MILLARD_LIVE_ViewONLY
WASATCH
UCIP_WASATCH_LIVE_ViewONLY
Updated  UCIP_WASATCH_LIVE_ViewONLY
BOX ELDER
UCIP_BOX ELDER_LIVE_ViewONLY
Updated  UCIP_BOX ELDER_LIVE_ViewONLY
WEBER
UCIP_WEBER_LIVE_ViewONLY
Updated  UCIP_WEBER_LIVE_ViewONLY
JUAB
UCIP_JUAB_LIVE_ViewONLY
Updated  UCIP_JUAB_LIVE_ViewONLY
SANPETE
UCIP_SANPETE_LIVE_ViewONLY
Updated  UCIP_SANPETE_LIVE_ViewONLY
WASHINGTON
UCIP_WASHINGTON_LIVE_ViewONLY
Updated  UCIP_WASHINGTON_LIVE_ViewONLY
GRAND
UCIP_GRAND_LIVE_ViewONLY
Updated  UCIP_GRAND_LIVE_ViewONLY
RICH
UCIP_RICH_LIVE_ViewONLY
Updated  UCIP_RICH_LIVE_ViewONLY
CARBON
UCIP_CARBON_LIVE_ViewONLY
Updated  UCIP_CARBON_LIVE_ViewONLY
DAGGETT
UCIP_DAGGETT_LIVE_ViewONLY
Updated  UCIP_DAGGETT_LIVE_ViewONLY
BEAVER
UCIP_BEAVER_LIVE_ViewONLY


### Update only one jurisdiction

In [8]:
import datetime

# Loop through the regional division to create the views or update existing views for ALL JURISDICTIONS
for index, county in enumerate(counties_munis):
    county_name = counties_munis.features[index].attributes['NAME']
    print(county_name)
    view_name = 'UCIP_' + county_name + "_LIVE_View"
    print(view_name)
    
    # Get the geometry for the regions
    view_geom = counties_munis.features[index].geometry.get('rings')
    name_avail = cm.is_service_name_available(service_name = view_name,service_type = "featureService")
    
    # Check if view exists
    if  view_name == 'UCIP_West Jordan_LIVE_View':
        print(view_name," Exists")
        
        # Search for View
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)
        update_date = datetime.date.fromtimestamp(view_search.modified/1000)
        print(update_date)
        print(datetime.datetime.today())
        
        if update_date == datetime.date.today():
            print("Recently Updated")
        else:

            service_layer = view_flc.layers[0]
            layerTags = [county_name,view_name,"UCIP","View Layer"]


            # Populate the update_dict with the geometry and the spatial reference
            dict_fields = [{"name":f"{f.name}","visible":True} for f in fields]
            #print(dict_fields)
            view_layer = view_search.get_data()
            view_layer["layers"][0]["popupInfo"]=popupInfo
            view_layer["layers"][0]["formInfo"]=formInfo
            view_layer["layers"][0]["layerDefinition"]["drawingInfo"]=drawingInfo

            #Update the layer tags and fields
            update_dict_fields = {"tags":layerTags,"fields":dict_fields}
            service_layer.manager.update_definition(update_dict_fields)
            view_search.update(update_dict_fields)

            # Update the layer symbology, pop-up info and form-info
            update_dict = {}
            update_dict = {"text":json.dumps(view_layer)}
            view_search.update(update_dict)

            # Assign category to new view layer
            service_layer_id=view_search.id
            my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                         "categories": ["/Categories/UCIP"]}}])

            # Add layer to UCIP_LIVE folder
            view_search.move(folder="UCIP - LIVE")

            print("Updated ",view_name)
    else:
        print("Not Today ",view_name,", not today.......")

print('Done!')


Nephi
UCIP_Nephi_LIVE_View
Not Today  UCIP_Nephi_LIVE_View , not today.......
Rockville
UCIP_Rockville_LIVE_View
Not Today  UCIP_Rockville_LIVE_View , not today.......
Woodruff
UCIP_Woodruff_LIVE_View
Not Today  UCIP_Woodruff_LIVE_View , not today.......
Sigurd
UCIP_Sigurd_LIVE_View
Not Today  UCIP_Sigurd_LIVE_View , not today.......
Cleveland
UCIP_Cleveland_LIVE_View
Not Today  UCIP_Cleveland_LIVE_View , not today.......
South Ogden
UCIP_South Ogden_LIVE_View
Not Today  UCIP_South Ogden_LIVE_View , not today.......
Wendover
UCIP_Wendover_LIVE_View
Not Today  UCIP_Wendover_LIVE_View , not today.......
Richmond
UCIP_Richmond_LIVE_View
Not Today  UCIP_Richmond_LIVE_View , not today.......
Riverdale
UCIP_Riverdale_LIVE_View
Not Today  UCIP_Riverdale_LIVE_View , not today.......
Brian Head
UCIP_Brian Head_LIVE_View
Not Today  UCIP_Brian Head_LIVE_View , not today.......
Oakley
UCIP_Oakley_LIVE_View
Not Today  UCIP_Oakley_LIVE_View , not today.......
New Harmony
UCIP_New Harmony_LIVE_View
N

### DEPRECATED: Create/Update Jurisdictional Views for Counties Only

In [54]:
# Loop through the regional division to create the views COUNTIES ONLY
import json
for index, county in enumerate(counties):
    county_name = counties.features[index].attributes['NAME']
    print(county_name)
    view_name = 'UCIP_' + county_name + "_All_LIVE_View"
    print(view_name)
    # Get the geometry for the regions
    view_geom = counties.features[index].geometry.get('rings')
    name_avail = cm.is_service_name_available(service_name = view_name,service_type = "featureService")
    # Check if view exists
    if  name_avail == True:
        new_view = ucip_locs_flc.manager.create_view(name=view_name)
        # Search for newly created View
        view_search = my_agol.content.search(view_name)[0]
        view_flc = FeatureLayerCollection.fromitem(view_search)

        service_layer = view_flc.layers[0]
        layerTags = [county_name,view_name,"UCIP","View Layer","Live"]


        
        # Populate the update_dict with the geometry and the spatial reference
        update_dict = {"tags":layerTags,"viewLayerDefinition":{"filter":
                                                               {"operator":"esriSpatialRelContains","value":
                                                                {"geometryType":"esriGeometryPolygon","geometry":
                                                                 {"rings": view_geom,
                                                                  "spatialReference":spat_ref}}}}}
        
        # Assign category to new view layer
        service_layer_id=view_search.id
        my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                     "categories": ["/Categories/UCIP"]}}])
        
        # Add layer to UCIP_LIVE folder
        

        # Update the definition to include the Area of Interest
        service_layer.manager.update_definition(update_dict)
        
        # Move layer to UCIP live folder
        view_search.move(folder="UCIP - LIVE")

        print("Added ",view_name)
    elif view_name == "UCIP_WASHINGTON_All_LIVE_View":
        view_search = my_agol.content.search(view_name)[0]
        print(view_search)
        view_flc = FeatureLayerCollection.fromitem(view_search)

        service_layer = view_flc.layers[0]
        layerTags = [county_name,view_name,"UCIP","View Layer","Live"]
        dict_fields = [{"name":f"{f.name}","visible":True} for f in fields]
        
        view_layer = view_search.get_data()
        #print(json.dumps(view_layer))
        view_layer["layers"][0]["popupInfo"]=popupInfo
        view_layer["layers"][0]["formInfo"]=formInfo
        view_layer["layers"][0]["layerDefinition"]["drawingInfo"]=drawingInfo
        print(json.dumps(view_layer))
        



        
        # Populate the update_dict with the geometry and the spatial reference
        #update_dict = {"tags":layerTags,"drawingInfo":ucip_symbology,"fields":dict_fields}
        update_dict = {}
        update_dict = {"text":json.dumps(view_layer)}
        
        # Assign category to new view layer
        service_layer_id=view_search.id
        my_agol.content.categories.assign_to_items(items = [{service_layer_id: {
                                                     "categories": ["/Categories/UCIP"]}}])
        
        # Add layer to UCIP_LIVE folder
        

        # Update the definition to include the Area of Interest
        #ucip_properties2 = ucip_properties
        #del ucip_properties2['editingInfo']
        #service_layer.manager.update_definition(update_dict)
        view_search.update(update_dict)
        
        # Move layer to UCIP live folder
        view_search.move(folder="UCIP - LIVE")

        print("Updated ",view_name)
    else:
        print("Exists")
        

CACHE
UCIP_CACHE_All_LIVE_View
Exists
DUCHESNE
UCIP_DUCHESNE_All_LIVE_View
Exists
DAVIS
UCIP_DAVIS_All_LIVE_View
Exists
MILLARD
UCIP_MILLARD_All_LIVE_View
Exists
WASATCH
UCIP_WASATCH_All_LIVE_View
Exists
BOX ELDER
UCIP_BOX ELDER_All_LIVE_View
Exists
WEBER
UCIP_WEBER_All_LIVE_View
Exists
JUAB
UCIP_JUAB_All_LIVE_View
Exists
SANPETE
UCIP_SANPETE_All_LIVE_View
Exists
WASHINGTON
UCIP_WASHINGTON_All_LIVE_View
<Item title:"UCIP_WASHINGTON_All_LIVE_View" type:Feature Layer Collection owner:GIS@UtahDEM>
{"tables": [], "layers": [{"layerDefinition": {"defaultVisibility": true, "drawingInfo": {"renderer": {"authoringInfo": {"classificationMethod": "esriClassifyManual", "visualVariables": [{"maxSliderValue": 68, "minSliderValue": 9, "theme": "high-to-low", "type": "sizeInfo"}]}, "type": "uniqueValue", "visualVariables": [{"type": "sizeInfo", "valueExpression": "// Write a script that returns a value that will be used to draw features.\r\n// For example, find the percentage of males:\r\n// Round(($

Exception: An unknown error occurred: Traceback (most recent call last):
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\gis\_impl\_con\_connection.py", line 1036, in post
    url=url, data=params, cert=cert, files=files, timeout=timeout
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\requests\sessions.py", line 590, in post
    return self.request('POST', url, data=data, json=json, **kwargs)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\requests\sessions.py", line 542, in request
    resp = self.send(prep, **send_kwargs)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\requests\sessions.py", line 655, in send
    r = adapter.send(request, **kwargs)
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\requests\adapters.py", line 449, in send
    timeout=timeout
  File "C:\Program Files\ArcGIS\Pro\bin\Pytho

### DEPRECATED: Test Definition to Update Layers

In [ ]:
import json

def update_layer_def(item):
    item_data = item.get_data()
    if item_data is not None:
        # Here note we are changing a specific part of the Layer Definition
        layer_def = item_data['layers'][3]['layerDefinition']
        print("*******************ORIGINAL DEFINITION*********************")
        print(json.dumps(item_data, indent=4, sort_keys=True))

        # Open JSON file containing symbology update
        with open('/path/to/drawingInfo.json') as json_data:
            data = json.load(json_data)

        # Set the drawingInfo equal to what is in JSON file
        layer_def['drawingInfo'] = data

        # Set the item_properties to include the desired update
        item_properties = {"text": json.dumps(item_data)}

        # 'Commit' the updates to the Item
        item.update(item_properties=item_properties)

        # Print item_data to see that changes are reflected
        new_item_data = item.get_data()
        print("***********************NEW DEFINITION**********************")
        print(json.dumps(new_item_data, indent=4, sort_keys=True))
    
    else:
        print("There is no Layer Definition at the moment..creating one...")
        create_layer_def(item)

In [27]:
# Get Master Layer Info

import arcgis
resturl=dict(ucip_locs)["url"]
layerurl=str(resturl)+"/"+str(0)
layer=arcgis.features.FeatureLayer(layerurl)

print(layerurl+"?f=pjson")
print(layer.properties)

https://services6.arcgis.com/KaHXE9OkiB9e63uE/arcgis/rest/services/UCIP_Master_Layer_081822_00865/FeatureServer/0?f=pjson
{
  "currentVersion": 11.2,
  "id": 0,
  "name": "UCIP_Testing_Survey",
  "type": "Feature Layer",
  "serviceItemId": "f3e43dcc7fc3495ea8731597b63b690d",
  "sourceSchemaChangesAllowed": true,
  "hasViews": true,
  "displayField": "assessor_name",
  "description": "",
  "copyrightText": "",
  "defaultVisibility": true,
  "ownershipBasedAccessControlForFeatures": {
    "allowOthersToQuery": true,
    "allowOthersToDelete": true,
    "allowOthersToUpdate": true,
    "allowAnonymousToQuery": true,
    "allowAnonymousToUpdate": false,
    "allowAnonymousToDelete": false
  },
  "editFieldsInfo": {
    "creationDateField": "CreationDate",
    "creatorField": "Creator",
    "editDateField": "EditDate",
    "editorField": "Editor"
  },
  "editingInfo": {
    "lastEditDate": 1700081788781,
    "schemaLastEditDate": 1700081788781,
    "dataLastEditDate": 1700081788781
  },
  "

In [ ]:
# Print County Names

for index, county in enumerate(counties):
    ind = len(counties_munis) - index - 1
    county_name = counties_munis.features[ind].attributes['NAME']
    if county_name[-4:] == "_ALL": 
        print(county_name)

In [ ]:
# Run this to delete all view layers 

if_delete = input("Do you want to delete all view layers?")
if if_delete == "yes":
    for index, county in enumerate(counties_munis):
        county_name = counties_munis.features[index].attributes['NAME']
        print(county_name)
        view_name = 'UCIP_' + county_name + "_LIVE_View"
        print(view_name)
        # Get the geometry for the regions
        view_geom = counties_munis.features[index].geometry.get('rings')
        name_avail = cm.is_service_name_available(service_name = view_name,service_type = "featureService")
        # Check if view exists
        if  name_avail == False:
            view_search = my_agol.content.search(view_name)[0]
            view_search.delete()
            print(view_name + " has been deleted.")

In [ ]:
# Backup as FGDB
from datetime import datetime
from arcgis.gis import GIS 
import arcpy 
import os 
import zipfile 
import glob 
import shutil 

current_time = datetime.now()
date = current_time.strftime('%m_%d_%y')

fgdb_title = ucip_locs.title 
fgdb_title=fgdb_title.replace('_', '')
fgdb_title=fgdb_title+"_"+date
print(fgdb_title)

root = r"C:\UCIP" 
#result = ucip_locs.export(fgdb_title, "File Geodatabase")
#items = gis.content.search(nombre)
result = ucip_locs.export(fgdb_title, 'File Geodatabase', parameters=None, wait='True') 
result.download(root)